In [1]:
import pickle
import os
import sys
from datetime import datetime
import threading
import math
import json
import torch
from transformers import *
import re
from nltk.corpus import stopwords
import urllib

### Embedding computation for Chinese text

In [29]:
def create_bert_emb(all_sents, tok_pooling='mean', get_cls_emb=False):
    if len(all_sents) > 0:
        with torch.cuda.device(0):
            all_toks = emb_tokenizer.batch_encode_plus(all_sents, padding='longest',\
                                                   add_special_tokens=True)
            tok_tensor = torch.tensor(all_toks['input_ids']).to('cuda')
            tok_tensor = tok_tensor[:, :512]
            with torch.no_grad():
                model_out = emb_model(tok_tensor)
                all_doc_tensor = model_out[0]
                if get_cls_emb:
                    all_doc_tensor = model_out[1]
                all_doc_tensor = all_doc_tensor.to('cpu')
            if get_cls_emb:
                return all_doc_tensor
            all_attn_mask = torch.tensor(all_toks['attention_mask'])
            ret_tensor = torch.FloatTensor(all_doc_tensor.size(0), all_doc_tensor.size(-1))
            for i in range(all_doc_tensor.size(0)):
                slen = torch.sum(all_attn_mask[i, :])
                if tok_pooling == 'mean':
                    ret_tensor[i, :] = torch.mean(all_doc_tensor[i, :slen, :], dim=0)
                elif tok_pooling == 'sum':
                    ret_tensor[i, :] = torch.sum(all_doc_tensor[i, :slen, :], dim=0)
                else:
                    return 'invalid tok pooling'
            return ret_tensor

In [30]:
def batchify(all_sents, batch_size=100):
    batches = []
    beg = 0
    end = batch_size
    while beg < len(all_sents):
        batches.append(all_sents[beg:end])
        beg = end
        end += batch_size
    return batches

In [31]:
emb_tokenizer_class = BertTokenizer
emb_tokenizer = emb_tokenizer_class.from_pretrained('hfl/chinese-bert-wwm')
with torch.cuda.device(0):
    with torch.no_grad():
        emb_model = BertModel.from_pretrained('hfl/chinese-bert-wwm',\
                                          output_hidden_states=False,\
                                          output_attentions=False)
        emb_model.eval()
        emb_model.to('cuda')

### Loading data

In [2]:
dir_path = '/homes/rpujari/scratch1_fortytwo/DARPA/'

In [3]:
metadata = json.load(open(dir_path + 'mpdd/metadata.json'))
dialogue = json.load(open(dir_path + 'mpdd/dialogue.json'))

In [33]:
all_utterances = []
utterance_ids = []
for conv_id in dialogue:
    conv = dialogue[conv_id]
    for i, turn in enumerate(conv):
#         for key in turn:
#             print(key, turn[key])
        all_utterances.append(turn['utterance'])
        utterance_ids.append(conv_id + '-' + str(i))
print(len(utterance_ids), len(all_utterances))

25548 25548


In [35]:
utterance_batches = batchify(all_utterances)
batch_utterance_embs = []
t1 = datetime.now()
for i, batch in enumerate(utterance_batches):
    batch_emb = create_bert_emb(batch)
    batch_utterance_embs.append(batch_emb)
    t2 = datetime.now()
    print(i + 1, '/', len(utterance_batches), 'done,', t2-t1)

1 / 256 done, 0:00:01.294949
2 / 256 done, 0:00:01.825791
3 / 256 done, 0:00:02.314426
4 / 256 done, 0:00:02.979115
5 / 256 done, 0:00:03.683817
6 / 256 done, 0:00:04.163849
7 / 256 done, 0:00:04.896527
8 / 256 done, 0:00:05.318308
9 / 256 done, 0:00:05.918438
10 / 256 done, 0:00:06.448533
11 / 256 done, 0:00:07.513016
12 / 256 done, 0:00:08.073557
13 / 256 done, 0:00:08.418241
14 / 256 done, 0:00:08.922635
15 / 256 done, 0:00:09.375076
16 / 256 done, 0:00:09.661677
17 / 256 done, 0:00:10.012492
18 / 256 done, 0:00:10.358761
19 / 256 done, 0:00:10.776029
20 / 256 done, 0:00:11.408135
21 / 256 done, 0:00:11.825479
22 / 256 done, 0:00:12.188088
23 / 256 done, 0:00:12.387972
24 / 256 done, 0:00:12.680954
25 / 256 done, 0:00:12.803513
26 / 256 done, 0:00:12.975283
27 / 256 done, 0:00:13.276735
28 / 256 done, 0:00:13.735419
29 / 256 done, 0:00:13.932247
30 / 256 done, 0:00:14.192142
31 / 256 done, 0:00:14.696530
32 / 256 done, 0:00:14.946954
33 / 256 done, 0:00:15.212822
34 / 256 done, 0:00

In [38]:
utterance_embs = torch.cat(batch_utterance_embs, dim=0)
print(utterance_embs.size())
with open(dir_path + 'mpdd/bert-base-utterance-embs.pkl', 'wb') as outfile:
    pickle.dump((utterance_embs, utterance_ids), outfile)

torch.Size([25548, 768])


### Translating to English
#### Argostranslate: https://www.argosopentech.com/argospm/index/
#### huggingface transformers

In [8]:
sent1 = '那個憨女人有什麼值得送的，正鵬這個人也真是的！'
sent2 = '哎喲，老婆子，你怎麼盡講那些不利於團結的話呢！他去送送他的同學也在情理之中嘛！'
sent3 = '爸、媽，我回來啦！'

In [15]:
from argostranslate import package, translate
package.install_from_path(dir_path + 'translate-zh_en-1_1.argosmodel')
installed_languages = translate.get_installed_languages()
print([str(lang) for lang in installed_languages])
translation_zh_en = installed_languages[1].get_translation(installed_languages[0])

### Original translations given in the paper

##### What is Zheng-Peng thinking? He has no need to send the silly woman home.
##### Hey. My old woman. How can you say such uncoordinated words? It’s reasonable for him to send his classmate home.”
##### Dad, Mom, I'm back!

#### Google translate webpage (API is billed and should be used via Google cloud platform)

##### What is there for that silly woman to give, and Zhengpeng is the real one!
##### Alas, old lady, how can you say all those things that are not good for unity! It makes sense for him to send off his classmates!
##### Dad, Mom, I'm back!

In [20]:
#Translations using argos-translate offline model
print(translation_zh_en.translate(sent1))
print(translation_zh_en.translate(sent2))
print(translation_zh_en.translate(sent3))

It was true that the female stereotyped had suffice, and that the perpetrators were.
Alexandre, How you can impose a boycott that is negative. He was sent to his fellows.
raz, I return!


#### Translations using hugging-face translation model

In [25]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

In [26]:
def translate_hf(chinese_sent):
    batch = tokenizer([chinese_sent], return_tensors="pt")
    generated_ids = model.generate(**batch)
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [14]:
print(translate_hf(sent1))
print(translate_hf(sent2))
print(translate_hf(sent3))

What's so good about that bitch, Jung-ho?
Honey, why are you saying things that are not good for unity? It's also reasonable for him to send his classmates!
Mom and Dad, I'm home!


### Tokenizing into sentences (Stanza: https://stanfordnlp.github.io/stanza/tokenize.html)

In [21]:
import stanza

In [22]:
stanza.download('zh-hant')

INFO:stanza:Downloading default packages for language: zh-hant (Traditional_Chinese)...
INFO:stanza:Finished downloading models and saved to /homes/rpujari/stanza_resources.


In [24]:
stanza_nlp = stanza.Pipeline(lang='zh-hant', processors='tokenize')
doc = stanza_nlp(sent2)
for i, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

INFO:stanza:Loading these models for language: zh-hant (Traditional_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsd     |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!


====== Sentence 1 tokens =======
id: (1,)	text: 哎喲
id: (2,)	text: ，
id: (3,)	text: 老
id: (4,)	text: 婆子
id: (5,)	text: ，
id: (6,)	text: 你
id: (7,)	text: 怎麼
id: (8,)	text: 盡講
id: (9,)	text: 那些
id: (10,)	text: 不利
id: (11,)	text: 於
id: (12,)	text: 團結
id: (13,)	text: 的
id: (14,)	text: 話
id: (15,)	text: 呢
id: (16,)	text: ！
====== Sentence 2 tokens =======
id: (1,)	text: 他
id: (2,)	text: 去
id: (3,)	text: 送送
id: (4,)	text: 他
id: (5,)	text: 的
id: (6,)	text: 同學
id: (7,)	text: 也
id: (8,)	text: 在
id: (9,)	text: 情理
id: (10,)	text: 之
id: (11,)	text: 中嘛
id: (12,)	text: ！
